In [1]:
import numpy as np
import os
import cv2
import time

In [2]:
def img_to_vox(img,VOXEL_SIZE,mask): 

    #first let's just get the minimum amount of coverage
    samples0 = int(img.shape[0] / float(VOXEL_SIZE)) + 4
    samples1 = int(img.shape[1] / float(VOXEL_SIZE)) + 4
    samples2 = int(img.shape[2] / float(VOXEL_SIZE)) + 4

    ixs0 = get_strides(samples0,img.shape[0],0,VOXEL_SIZE)
    ixs1 = get_strides(samples1,img.shape[1],0,VOXEL_SIZE)
    ixs2 = get_strides(samples2,img.shape[2],0,VOXEL_SIZE)
    
    subvoxels = []
    locations = []
    centroids = []
    for i0,x0 in enumerate(ixs0):
        for i1,x1 in enumerate(ixs1):
            for i2,x2 in enumerate(ixs2):
                if mask[x0:x0+VOXEL_SIZE,x1:x1+VOXEL_SIZE,x2:x2+VOXEL_SIZE].mean() > .99:
                    #basically no lung in this voxel, might as well ignore.
                    continue
                subvoxels.append(img[x0:x0+VOXEL_SIZE,x1:x1+VOXEL_SIZE,x2:x2+VOXEL_SIZE])
                assert subvoxels[-1].shape == (VOXEL_SIZE,VOXEL_SIZE,VOXEL_SIZE), 'bad subvoxel shape ' + str(subvoxels[-1].shape) + ' ' + str([x0,x1,x2]) + ' ' + str(img.shape)
                locations.append((i0,i1,i2))
                centroids.append((x0+VOXEL_SIZE/2,x1+VOXEL_SIZE/2,x2+VOXEL_SIZE/2))
    X = np.stack(subvoxels, axis=0)
    X = np.expand_dims(X, 1)
    #normalized locations
    #allows us to de-weight certain places...

    return X,locations,centroids

In [3]:
def crop_img(img_cpy):
    # img_raw = np.load(patient)
    # downsample = 1
    masks = []
    img_raw = img_cpy.copy()

    for i in range(img_raw.shape[2]):
        img_slice = img_raw[ :,:,i]
        img = img_slice.copy()

        img[img>-300] = 255
        img[img<-300] = 0
        img = np.uint8(img)
        _,contours,_ = cv2.findContours(img,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        if len(contours) > 0:


            largest_contour = max(contours, key=cv2.contourArea)
        else:
            mask = (np.zeros(img.shape, np.uint8) < 255)
            masks.append(mask)
            continue
        mask = np.zeros(img.shape, np.uint8)
        cv2.fillPoly(mask, [largest_contour], 255)

    #		 imshow(mask); show()

        # apply mask to threshold image to remove outside. this is our new mask
        img = ~img 

        img[(mask == 0)] = 0 # <-- Larger than threshold value


        # apply closing to the mask
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
        img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)  # <- to remove speckles...
        img = cv2.morphologyEx(img, cv2.MORPH_DILATE, kernel)
        img = cv2.morphologyEx(img, cv2.MORPH_DILATE, kernel)
        img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
        img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
        img = cv2.morphologyEx(img, cv2.MORPH_ERODE, kernel)
        img = cv2.morphologyEx(img, cv2.MORPH_ERODE, kernel)

        #the image has an outside part which we don't care about (value 0)
        #and a boundary that we don't care about (value 255)
        #and some noise that we don't care about (value 125)
        mask = (img < 255)
    #		 img_raw[~mask] = -2000
    #		 imshow(img_raw); colorbar(); show()
        masks.append(mask)

    #now we have one mask per slice. To determine our bounding box, take the max x,y,z plus a fuzz factor
    ixs_to_remove = [i for i,m in enumerate(masks) if np.mean(m) > .995]

    # masks =[m for m in masks if np.mean(m) < .995]
    masks = np.stack(masks, axis=2)
    masks = np.delete(masks, ixs_to_remove, axis=2)
    img_raw = np.delete(img_raw, ixs_to_remove, axis=2)


    #0 = mask, 1 = background
    x_dim = np.min(masks, axis=(1,2))
    y_dim = np.min(masks, axis=(0,2))
    z_dim = np.min(masks, axis=(0,1))

    xstart = find_start(1 - x_dim, .5)
    xend = -(find_start(1 - x_dim[::-1], .5) + 1)

    ystart = find_start(1 - y_dim, .5)
    yend = -(find_start(1 - y_dim[::-1], .5) + 1)


    zstart = find_start(1 - z_dim, .5)
    zend = -( find_start(1 - z_dim[::-1], .5) + 1)

    # try:
    assert xstart < int(img_raw.shape[0]*.5) < img_raw.shape[0] - xend, 'bad crop ' + str(xstart) + ' ' + str(xend) + ' ' + str(img_raw.shape[0])
    assert ystart < int(img_raw.shape[1]*.5) < img_raw.shape[1] - yend, 'bad crop ' + str(ystart) + ' ' + str(yend) + ' ' + str(img_raw.shape[1])
    assert zstart < int(img_raw.shape[2]*.5) < img_raw.shape[2] - zend, 'bad crop ' + str(zstart) + ' ' + str(zend) + ' ' + str(img_raw.shape[2])
    assert xend < 0 and yend < 0 and zend < 0, 'one end >= 0'
    assert xstart >= 0 and ystart >= 0 and zstart >= 0, 'one start <= 0'
    # except AssertionError as e:
        # print 'WARNING cropping failed. using full img', e
        # return img_raw

    return img_raw[xstart:xend,ystart:yend,zstart:zend], masks[xstart:xend,ystart:yend,zstart:zend]


In [4]:
def find_start(arr, thresh=.5):
    #determine when the arr first exceeds thresh
    #arr = arr.ravel()
    for i in range(arr.shape[0]):
        if arr[i] > thresh:
            #print 'returning', i
            return np.clip(i - 8, 0, arr.shape[0])
    return 0

In [5]:
def get_strides(steps,size,offset,VOXEL_SIZE):
    if steps * VOXEL_SIZE < size - 2*offset:
        #not enough coverage. start and end are modified
        start = (size - steps*VOXEL_SIZE) / 2
        end = size - start - VOXEL_SIZE
    else:
        start = offset
        end = size-VOXEL_SIZE - offset
    return list(np.around(np.linspace(start,end,steps)).astype('int32'))

In [6]:
patients = [f.replace('.npy','') for f in os.listdir('../data/stage2_arrays/') if '.npy' in f]

start = time.time()
for patient in patients:
    xorig = np.load('../data/stage2_arrays/'+patient+'.npy')
    shape = xorig.shape
    
    x,mask = crop_img(xorig)
    
    x = ((x + 1000.) / (400. + 1000.)).astype('float32')
    voxels, locs, centroids = img_to_vox(x,64,mask)
    
    np.savez_compressed('../data/stage2_voxels_mask/'+patient+'.npz',vox=voxels,locs=locs,cents=centroids,shape=shape)
    
    if patients.index(patient)%20==0:
        print ('Processed..',patients.index(patient),'out of',len(patients),'in',time.time()-start)
        start = time.time()
        print ('\tVoxels shape',voxels.shape)
        print ('\tLocs shape',len(locs))
        print ('\tCentroids shape',len(centroids))
        print ('\tPatient shape',xorig.shape)
        

Processed.. 0 out of 210 in 23.02389359474182
	Voxels shape (385, 1, 64, 64, 64)
	Locs shape 385
	Centroids shape 385
	Patient shape (298, 298, 245)
Processed.. 20 out of 210 in 538.1122269630432
	Voxels shape (449, 1, 64, 64, 64)
	Locs shape 449
	Centroids shape 449
	Patient shape (464, 464, 344)
Processed.. 40 out of 210 in 645.5356702804565
	Voxels shape (467, 1, 64, 64, 64)
	Locs shape 467
	Centroids shape 467
	Patient shape (350, 350, 320)
Processed.. 60 out of 210 in 639.7076284885406
	Voxels shape (441, 1, 64, 64, 64)
	Locs shape 441
	Centroids shape 441
	Patient shape (380, 380, 346)
Processed.. 80 out of 210 in 692.105610370636
	Voxels shape (446, 1, 64, 64, 64)
	Locs shape 446
	Centroids shape 446
	Patient shape (308, 308, 305)
Processed.. 100 out of 210 in 669.0358333587646
	Voxels shape (399, 1, 64, 64, 64)
	Locs shape 399
	Centroids shape 399
	Patient shape (414, 414, 314)
Processed.. 120 out of 210 in 653.323878288269
	Voxels shape (332, 1, 64, 64, 64)
	Locs shape 332
	Ce